This notebook allow you to print results of models in tables

In [1]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from twinews.utils import *
from twinews.evaluation.utils import *

In [3]:
if False:
    purgeSubsampledRankings()

In [25]:
if False:
    removeRankingsAndScores('tfidf-b0e81')

In [5]:
def basicPrint(model, **kwargs):
    printReportKwargs = \
    {
        'model': model,
        'splitVersion': 2,
        'noSubsampling': True,
        'metaFilter': \
        {
            # "implementation": "gensim-lda",
        },
        'metricsFilter': {'ndcg'}, # None, {'ndcg', 'ndcg@10'}
        'sortBy': 'ndcg',
    }
    printReportKwargs = mergeDicts(printReportKwargs, kwargs)
    printReport(**printReportKwargs)

In [13]:
if False:
    twinewsRankings = getTwinewsRankings()
    keysToDelete = set()
    for key in twinewsRankings.keys():
        if "combin" in key:
            meta = twinewsRankings.getMeta(key)
            hasTFIDF = False
            for m in meta['models']:
                if 'tfidf' in m:
                    hasTFIDF = True
                    break
            if hasTFIDF and meta['splitVersion'] == 1:
                keysToDelete.add(key)
    print(keysToDelete)

{'combin-06486', 'combin-1c882', 'combin-c9868', 'combin-66946', 'combin-a9696', 'combin-484c1', 'combin-67d64', 'combin-6631d', 'combin-76f78', 'combin-eeae6', 'combin-44cdf', 'combin-98b76', 'combin-bd3c8', 'combin-943dd', 'combin-80293', 'combin-f9e6a', 'combin-805b7', 'combin-91075', 'combin-c8f2c', 'combin-95c57', 'combin-57ef6', 'combin-b6c2e', 'combin-334a8', 'combin-77e03'}


In [15]:
if False:
    for key in keysToDelete:
        removeRankingsAndScores(key)

# Model lists per split version

In [6]:
def printModelList(splitVersion):
    twinewsRankings = getTwinewsRankings()
    models = set()
    for key in twinewsRankings.keys():
        meta = twinewsRankings.getMeta(key)
        if meta["splitVersion"] == splitVersion:
            models.add(meta['model'])
    print(" ".join(sorted(list(models))) + " (" + str(len(models)) + ")")

In [7]:
printModelList(2)

BM25_yf bert bm25 combin dbert-base dbert-ft doc2vec ideal infersent lda nmf random sent2vec stylo tfidf usent word2vec worst (18)


In [8]:
printModelList(1)

bert bm25 combin dbert-base dbert-ft doc2vec ideal infersent lda nmf random sent2vec stylo tfidf usent word2vec worst (17)


# Reference models

In [ ]:
splitVersion = 2
df = printReport(model='ideal', splitVersion=splitVersion, noSubsampling=True, sortBy='ndcg')
df = printReport(model='random', splitVersion=splitVersion, noSubsampling=True, sortBy='ndcg')
df = printReport(model='worst', splitVersion=splitVersion, noSubsampling=True, sortBy='ndcg')

# Ad-hoc models

In [ ]:
basicPrint('bm25')

In [ ]:
basicPrint('BM25_yf', noSubsampling=False)

# Models that use vector representations

In [ ]:
basicPrint('lda')

In [30]:
basicPrint('nmf')

These values are common to all rows (19):

	- nmfAlpha: 0.1
	- model: nmf
	- splitVersion: 2
	- maxDF: 300
	- implementation: sklearn-nmf
	- distance: cosine
	- minDF: 0.0005
	- maxUsers: None
	- useExtraNews: False
	- lowercase: True


,id,doLemmatization,historyRef,maxDocuments,maxIter,nbTopics,nmfInit,nmfL1Ratio,ndcg
12,nmf-9cd4f,False,30,100000,200,100,nndsvd,0,0.49322
17,nmf-330c8,False,30,10000,200,100,nndsvd,0,0.48901
10,nmf-b1354,False,30,100000,200,100,None,0,0.48842
4,nmf-ee6c6,False,1.0,100000,200,100,nndsvd,0,0.48682
2,nmf-ad87b,False,30,100000,400,100,nndsvd,0,0.48653
0,nmf-84f4b,False,0.6,100000,200,100,nndsvd,0,0.48336
14,nmf-054a5,False,10,100000,200,100,nndsvd,0,0.4805
16,nmf-87941,False,0.3,100000,400,100,nndsvd,0,0.47088
5,nmf-b286d,False,0.3,300000,500,100,nndsvd,0,0.46988
6,nmf-2d0d6,False,0.3,300000,400,100,nndsvd,0,0.46935


In [31]:
basicPrint('tfidf')

Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
Found key difference: {'maxIter'}
These values are common to all rows (21):

	- model: tfidf
	- splitVersion: 2
	- maxUsers: None
	- distance: cosine
	- sublinearTF: True


,id,dimensions,doLemmatization,historyRef,lowercase,maxDF,maxIter,minDF,ndcg
4,tfidf-be107,nan,False,0.4,False,300,N/A,0.0005,0.54867
15,tfidf-6c83e,nan,False,0.35,False,300,N/A,0.0005,0.54863
17,tfidf-01001,nan,False,0.3,False,300,N/A,0.0005,0.5484
0,tfidf-7c894,nan,False,0.45,False,300,N/A,0.0005,0.54769
20,tfidf-f57a2,nan,False,0.5,False,300,N/A,0.0005,0.54631
13,tfidf-11f90,nan,False,0.55,False,300,N/A,0.0005,0.54509
1,tfidf-9bb25,nan,False,0.6,False,300,N/A,0.0005,0.54388
9,tfidf-e85ec,nan,False,0.2,False,300,N/A,0.0005,0.54294
14,tfidf-6056f,nan,False,30,False,300,N/A,0.0005,0.53276
3,tfidf-487b6,3000,False,0.3,False,300,10,0.0005,0.53182


In [ ]:
basicPrint('dbert-ft')

In [ ]:
basicPrint('dbert-base')

In [ ]:
basicPrint('stylo')

In [ ]:
basicPrint('infersent')

In [ ]:
basicPrint('sent2vec')

In [ ]:
basicPrint('word2vec')

In [ ]:
basicPrint('doc2vec')

In [ ]:
basicPrint('usent')

In [ ]:
basicPrint('bert')

# Combinations of models

In [29]:
# Expected 156 rows:
basicPrint('combin')

These values are common to all rows (156):

	- alphas: [0.5, 0.5]
	- model: combin
	- splitVersion: 2
	- maxUsers: None
	- weights: [0.5, 0.5]
	- betas: ['LOG', 'LOG']


,id,models,rankAsScore,ndcg
137,combin-cf91b,"['bm25-1eb2a', 'dbert-ft-d1b5f']","[False, False]",0.58772
138,combin-c56e2,"['dbert-ft-d1b5f', 'tfidf-be107']","[False, False]",0.58034
61,combin-f3aa7,"['bm25-1eb2a', 'doc2vec-19ca3']","[False, False]",0.57265
117,combin-e7de3,"['bm25-1eb2a', 'tfidf-be107']","[False, False]",0.56678
107,combin-84cca,"['bm25-1eb2a', 'tfidf-be107']","[True, True]",0.56655
150,combin-6b411,"['dbert-ft-d1b5f', 'doc2vec-19ca3']","[False, False]",0.56448
25,combin-a9c56,"['bm25-1eb2a', 'doc2vec-19ca3']","[True, True]",0.56408
99,combin-01199,"['doc2vec-19ca3', 'tfidf-be107']","[False, False]",0.56279
83,combin-1e53b,"['bm25-1eb2a', 'dbert-base-aafd1']","[False, False]",0.5598
8,combin-7d8fe,"['bm25-1eb2a', 'word2vec-4a2c5']","[False, False]",0.55949


# Deep Learning models

# Reports on the test set (split version 1)

In [10]:
splitVersion = 1

In [ ]:
report = printReport(splitVersion=splitVersion, model='ideal')
report = printReport(splitVersion=splitVersion, model='random')
report = printReport(splitVersion=splitVersion, model='worst')

In [27]:
report = printReport\
(
    splitVersion=splitVersion,
    excludedModels={'combin', 'ideal', 'worst', 'random'},
    # metricsFilter={'ndcg'},
    sortBy='ndcg',
    noSubsampling=True,
    onlyFields={'id', 'model'}
)

,id,model,map,mrr,ndcg,ndcg@10,ndcg@100,p@10,p@100
9,bm25-933f7,bm25,0.36316,0.61723,0.60819,0.44273,0.53604,0.31301,0.07674
8,tfidf-4a1dc,tfidf,0.34771,0.60442,0.59695,0.42603,0.52076,0.29923,0.07479
5,doc2vec-e013a,doc2vec,0.3232,0.58795,0.58202,0.40184,0.50388,0.28369,0.07424
7,nmf-6078e,nmf,0.26979,0.51627,0.54098,0.33832,0.4599,0.24526,0.07211
3,dbert-ft-7847a,dbert-ft,0.27155,0.53054,0.53664,0.3472,0.44411,0.24932,0.06864
1,usent-64ec0,usent,0.24835,0.49151,0.5172,0.31744,0.41832,0.22694,0.06454
0,lda-82272,lda,0.19536,0.40872,0.47486,0.24493,0.37615,0.18029,0.06437
12,dbert-base-d092a,dbert-base,0.18273,0.43752,0.46352,0.25297,0.34227,0.17994,0.05452
2,infersent-77ec7,infersent,0.18162,0.42212,0.46136,0.246,0.34086,0.17558,0.05449
11,bert-a7c9a,bert,0.152,0.37992,0.43154,0.21174,0.29781,0.15102,0.04855


In [28]:
# Expected 156 rows:
report = printReport\
(
    splitVersion=splitVersion,
    model='combin',
    # metricsFilter={'ndcg'},
    sortBy='ndcg',
    noSubsampling=True,
)

These values are common to all rows (144):

	- alphas: [0.5, 0.5]
	- model: combin
	- splitVersion: 1
	- maxUsers: None
	- weights: [0.5, 0.5]
	- betas: ['LOG', 'LOG']


,id,models,rankAsScore,map,mrr,ndcg,ndcg@10,ndcg@100,p@10,p@100
93,combin-8f846,"['bm25-933f7', 'dbert-ft-7847a']","[False, False]",0.39517,0.66123,0.63565,0.47918,0.57028,0.3363,0.07999
113,combin-6ecf3,"['dbert-ft-7847a', 'tfidf-4a1dc']","[False, False]",0.38256,0.6522,0.62673,0.46738,0.5582,0.32716,0.07858
76,combin-6fad8,"['bm25-933f7', 'doc2vec-e013a']","[False, False]",0.37663,0.63976,0.62146,0.45809,0.55266,0.32122,0.07817
100,combin-efad4,"['bm25-933f7', 'tfidf-4a1dc']","[False, False]",0.37163,0.63546,0.61633,0.45405,0.54474,0.31846,0.07679
23,combin-1c964,"['bm25-933f7', 'doc2vec-e013a']","[True, True]",0.36506,0.6335,0.61391,0.44685,0.54347,0.31323,0.07748
17,combin-35594,"['doc2vec-e013a', 'tfidf-4a1dc']","[False, False]",0.36301,0.62302,0.61083,0.44252,0.53976,0.31062,0.07715
109,combin-d0fd0,"['dbert-ft-7847a', 'doc2vec-e013a']","[False, False]",0.3584,0.62854,0.61009,0.44044,0.54034,0.31016,0.07831
8,combin-904f0,"['bm25-933f7', 'word2vec-f3c38']","[False, False]",0.36316,0.61723,0.60819,0.44273,0.53604,0.31301,0.07674
114,combin-50f90,"['bm25-933f7', 'dbert-base-d092a']","[False, False]",0.36089,0.6235,0.60802,0.44445,0.53385,0.31292,0.07555
88,combin-959fb,"['bm25-933f7', 'nmf-6078e']","[False, False]",0.35361,0.62588,0.60763,0.43587,0.53769,0.30533,0.07742
